In [ ]:
import pandas as pd
import matplotlib as plt
%matplotlib inline

In [393]:
dataSet = pd.read_csv('all_data.csv',sep=';',encoding='utf-8')

In [ ]:
dataSet.head()

In [394]:
dataSet = dataSet.drop(columns={'Unnamed: 0','author','author_type','deal_type','city','link','accommodation_type','phone','residential_complex'},axis=1)
dataSet = dataSet[(dataSet.year_of_construction > 0)]
dataSet = dataSet[(dataSet.living_meters > 0)]
dataSet = dataSet[(dataSet.kitchen_meters > 0)]

dataSet.dropna()

In [395]:
dataSet.head()

,Unnamed: 0,author,author_type,link,city,deal_type,accommodation_type,floor,floors_count,rooms_count,...,price_per_m2,price,year_of_construction,living_meters,kitchen_meters,phone,district,street,underground,residential_complex
1,1,White and Broughton,real_estate_agent,https://www.cian.ru/sale/flat/280986757/,Москва_и_др.,sale,flat,17,41,1,...,618811,25000000,2021,15.2,9.1,79646331510,Хорошевский,Ленинградский проспект,Аэропорт,Прайм Парк
3,3,White and Broughton,real_estate_agent,https://www.cian.ru/sale/flat/283638079/,Москва_и_др.,sale,flat,9,41,1,...,617283,25000000,2022,17.3,9.3,79646331510,Хорошевский,Ленинградский проспект,Аэропорт,Прайм Парк
5,5,Времена года,real_estate_agent,https://www.cian.ru/sale/flat/285378586/,Москва_и_др.,sale,flat,6,13,1,...,983122,69900000,2019,37.1,12.8,79152007823,Хамовники,Усачева,Спортивная,Садовые кварталы
7,7,Century 21 Premium Property,real_estate_agent,https://www.cian.ru/sale/flat/287489122/,Москва_и_др.,sale,flat,33,36,1,...,626666,47000000,2020,35.0,20.0,79175213146,Хорошево-Мневники,Шелепихинская набережная,Шелепиха,Сердце Столицы
9,9,Whitewill,real_estate_agent,https://www.cian.ru/sale/flat/286402225/,Москва_и_др.,sale,flat,6,41,1,...,724390,29700000,2022,24.0,6.0,79165694109,Хорошевский,Ленинградский проспект,Аэропорт,Прайм Парк


In [ ]:
# dataSet.info()
dataSet.plot(title='Data Visualize',kind='line')